In [ ]:
import pandas as pd
import xlrd
import numpy as np
from ipywidgets import *
from ipywidgets import interact,fixed
import pdb
import matplotlib.pyplot as plt

import plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 150)
pd.options.display.max_rows = 150

# read data from excel 

In [ ]:
data = pd.read_excel('Daten_f_viesha.xlsx',sheet_name='DATENMITTAMB',parse_dates=True,index_col='Datum')
lookup = pd.read_excel('Daten_f_viesha.xlsx',sheet_name='lookup')
lookup['Kategorie'] = lookup.Kategorie.fillna(method='ffill')
#data.set_index(['Datum'])
data.describe()

# null values?

In [ ]:
data.isnull().sum()

# drop null value columns

In [ ]:
exclude_cols = ['Feuchte','IGT','PPS','Spaltfestigkeit','Sampler status','Tambour','Tambournummer']
data = data.drop(exclude_cols, axis=1)
lookup = lookup[~lookup.Parameter.isin(exclude_cols)]
data['goodness'] = (0.5*data['Steifigkeit längs'] + data['Steifigkeit quer']) / data['L/Q-Verhältnis']
data['Shive long to total'] = data['Shive amount, long']/data['Shive amount, total']*100
data['Shive wide to total'] = data['Shive amount, wide']/data['Shive amount, total']*100

In [ ]:
row_filter = pd.DataFrame(data.groupby(['Sorte'])['Sorte'].count())
row_filter = row_filter[row_filter.Sorte >= 150]

data_filter = data[data.Sorte.isin(row_filter.index.values)]

print('-Number of rows raw input: ' + str(len(data)))
print('-Number of rows filtered input: ' + str(len(data_filter)))
print('---Number of unique Sorte: ' + str(len(data.Sorte.unique())))
print('---Number of filtered unique Sorte (>= 150rows): ' + str(len(data_filter.Sorte.unique())))

# Correlations?

In [ ]:
def plot_pearson_correlation(data=None,lookup=None, Sorte=None, target_col = 'Dicke'):
    if not (Sorte is None):
        df = data[data.Sorte==Sorte]
    else:
        df = data
    
    x_cols = [col for col in df.columns if col in lookup]
              #['Verhältnis HS in MB Einlage','Geschwindigkeit KM m/min','Zug SSW m/min',
              #'Zug 1.-2. Presse m/min','Zug 2.-3. Presse m/min ','Zug 3. Presse - VTG m/min']]
    labels = []
    values = []
    for col in x_cols:
        labels.append(col)
        values.append(np.corrcoef(df[col].values, df[target_col].values)[0,1])
    ind = np.arange(len(labels))
    width = 0.9
    fig, ax = plt.subplots(figsize=(10,10))
    plt.grid(True)
    rects = ax.barh(ind, np.array(values), color='y')
    ax.set_yticks(ind+((width)/2.))
    ax.set_yticklabels(labels, rotation='horizontal')
    ax.set_xlabel("Correlation coefficient")
    ax.set_title("Correlation coefficient")
    #autolabel(rects)
    plt.show()
    

def plot_timeseries(data=None,Sorte=None,lookup=None,clip=False,file=None,cluster=None):
    if not (Sorte is None):
        df = data[data.Sorte==Sorte]
    else:
        df = data
        
    x_cols = [col for col in df.columns if col in lookup]
    fig = tools.make_subplots(rows=1, cols=1)
    for col in x_cols:
        y = df[col]
        mean = y.mean()
        sd = y.std()
        lower = mean-0.5*(sd)
        upper = mean+0.5*(sd)
        #print('lower: ' + str(lower) + ', upper: ' + str(upper))
        if clip==True:
            y.clip(lower,upper,axis = 1,inplace = True)
        df[col] = y
    if not (cluster is None):
        df = data[data.Kmeans_cluster==cluster]
    cf.set_config_file(offline=True, world_readable=True)
    df = df[x_cols]
    df.iplot(kind='scatter',dimensions=(2200, 650),filename=file,title=file,xTitle='Date',yTitle='fraction') 

In [ ]:
pd.DataFrame(data = ['Shive long to total'])

# Plot Faserdaten non-Fractions over time

In [ ]:
include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid','Shive amount, total', 'Shive amount, long','Shive amount, wide']
include = include[~include.Parameter.isin(exclude)]
include = include[~include.Parameter.str.contains('fraction')]
include.append(pd.DataFrame( ['Shive long to total']),ignore_index=True)
include.append(pd.DataFrame( ['Shive wide to total']),ignore_index=True)
include
#plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="NonFraction")

# Plot Faserdaten Fractions over time

In [ ]:
include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 1')]
include = include[~include.Parameter.str.contains('Shiv')]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Fraction1")

include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 2')]
include = include[~include.Parameter.str.contains('Shiv')]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Fraction2")

include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 3')]
include = include[~include.Parameter.str.contains('Shiv')]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Fraction3")

include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 4')]
include = include[~include.Parameter.str.contains('Shiv')]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Fraction4")

include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 5')]
include = include[~include.Parameter.str.contains('Shiv')]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Fraction5")

# Plot Labordaten Steifigkeit & LQV over time

In [ ]:
include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Laborergebnisse'])
exclude = ['Dicke','Flächengewicht','Feuchte','IGT','PPS','Spaltfestigkeit']
include = include[~include.Parameter.isin(exclude)]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Laborergebnisse_Steifigkeit")

plot_timeseries(data=data_filter, lookup = 'L/Q-Verhältnis',clip=False,file="Laborergebnisse_LQV")

# plot Dimensions over time

In [ ]:
include = ['Dicke','Flächengewicht']
plot_timeseries(data=data_filter, lookup = include,clip=False,file="Laborergebnisse_Dicke_Gewicht")

# Plot normalized Maschinendaten

In [ ]:
include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Maschinenparameter'])
exclude = ['Verhältnis HS in MB Einlage','Geschwindigkeit KM m/min']
include = include[~include.Parameter.isin(exclude)]
plot_timeseries(data=data_filter, lookup = include.values,clip=False,file="Maschine")

# Korrelation Maschinendaten & Faserdaten

In [ ]:
lookup_Maschine = lookup.Parameter[lookup.Kategorie=='Maschinenparameter'].values
interact(plot_pearson_correlation
         , data = fixed(data_filter)
         , lookup = fixed(lookup_Maschine)
         , Sorte=data.Sorte.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Faserdaten']);

# Korrelation Maschinendaten-Laborergebnisse

In [ ]:
interact(plot_pearson_correlation
         , data = fixed(data_filter)
         , lookup = fixed(lookup_Maschine)
         , Sorte=data.Sorte.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Laborergebnisse']);

# Korrelation Faserdaten-Laborergebnisse

In [ ]:
lookup_Faser = lookup.Parameter[lookup.Kategorie=='Faserdaten'].values
interact(plot_pearson_correlation
         , data = fixed(data_filter.dropna())
         , lookup = fixed(lookup_Faser)
         , Sorte=data.Sorte.unique()
         , target_col= lookup.Parameter[lookup.Kategorie=='Laborergebnisse']);

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
def bench_k_means(estimator, name, data):
    estimator.fit(data)

In [ ]:
exclude_cols = ['Sorte','Sorte2','Grammatur','Tambour','Geschwindigkeit KM m/min','BatchId','Shive batchid','Tambournummer']
data_final = data_filter.drop(exclude_cols, axis=1).fillna(method = 'bfill')
for x in data_final.columns:
    data_final[x] = pd.to_numeric(data_final[x], errors='coerce')
data_final.describe()

In [ ]:
kmeans = KMeans(n_clusters=5)  
kmeans.fit(data_final)  
#print(kmeans.cluster_centers_)  
data_final['Kmeans_cluster'] = kmeans.labels_
data_final.describe()

In [ ]:
include = pd.DataFrame(lookup.Parameter[lookup.Kategorie=='Faserdaten'])
exclude = ['BatchId','Shive batchid']
include = include[~include.Parameter.isin(exclude)]
include = include[include.Parameter.str.contains('fraction 4')]
include = include[~include.Parameter.str.contains('Shiv')]

#plot_timeseries(data=data_filter, lookup = include.values,clip=False, cluster=3, file="Maschine")

interact(plot_timeseries
         , data = fixed(data_final)
         , lookup = fixed(include)
         , cluster = 3);